In [102]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
"""
用于设置策略参数并执行策略
"""

import pandas as pd
import pybroker as pb
from pybroker import Strategy
from pybroker.ext.data import AKShare
from data.data_resouce import AKShare_ETF, AKShare_ZHISHU
from strategy.strategy_content import buy_with_indicator
from strategy.strategy_chart import create_strategy_charts
from indicator.indicator_talib import calculate_indicator
import akshare as ak
    
# 定义全局参数 "stock_code"（股票代码）
# pb.param(name='stock_code', value='300747') 
# pb.param(name='stock_code', value='002230') 
pb.param(name='stock_code', value='000852') 
# 定义全局参数 "start_date" 开始日期
pb.param(name='dt_start_date', value='20170801') 
# 定义全局参数 "end_date" 结束日期
pb.param(name='dt_end_date', value='20241101') 
# 定义全局参数 "start_date" 开始日期
pb.param(name='bt_start_date', value='20180802') 
# 定义全局参数 "end_date" 结束日期
pb.param(name='bt_end_date', value='20241101') 

# 定义全局参数 "percent"（持仓百分比） 1代表100% 0.25代表25%
pb.param(name='percent', value=1)
# 定义全局参数 "stop_loss_pct"（止损百分比）
pb.param(name='stop_loss_pct', value=10)
# 定义全局参数 "stop_profit_pct"（止盈百分比）
pb.param(name='stop_profit_pct', value=10)

10

In [104]:
# 创建策略配置，初始资金为 500000
from data.data_resouce import AKShare_ALL

my_config = pb.StrategyConfig(initial_cash=500000,fee_amount=0.0013,fee_mode = pb.common.FeeMode.ORDER_PERCENT)
#使用AKShare获取股票OHLCV数据
akshare = AKShare_ALL()
df = akshare.query(symbols=[pb.param(name='stock_code')], start_date=pb.param(name='dt_start_date'), end_date=pb.param(name='dt_end_date'),adjust="hfq")
#使用at-lib计算并获取指标
#计算MACD参数
data_with_indicator = calculate_indicator(df)
print("打印技术指标")
print(data_with_indicator)
#macd_data = calculate_macd(df)
#存入数据库
#save_data_macd(macd_data)

# 使用配置、数据源、起始日期、结束日期，以及刚才定义的交易策略创建策略对象
#strategy = Strategy(akshare, start_date='20230101', end_date='20230201', config=my_config)
# 添加执行策略，设置股票代码和要执行的函数

#在pybroker中注册指标名称
'''MACD指标'''
pb.register_columns('macd')
pb.register_columns('macdsignal')
pb.register_columns('macdhist')
pb.register_columns('sma_short')
pb.register_columns('sma_long')
pb.register_columns('sma250')

'''MFI指标'''
pb.register_columns('MFI')
pb.register_columns('SMA250')

#创建策略
strategy = Strategy(data_with_indicator, start_date=pb.param(name='bt_start_date'), end_date=pb.param(name='bt_end_date'), config=my_config)
#配置策略执行参数
strategy.add_execution(buy_with_indicator, symbols=[pb.param(name='stock_code')])

# 执行回测，并打印出回测结果的度量值（四舍五入到小数点后四位）
result = strategy.backtest()
print('======基准收益率==========================')
start_open = df[df['date'] == pb.param(name='bt_start_date')]["open"].values[0]
end_open = df[df['date'] == pb.param(name='bt_end_date')]["open"].values[0]
last_open = df['open'].iloc[-1]
benchmark_return = (end_open - start_open) / start_open * 100
print(f"基准收益率{benchmark_return}")
# df=result.metrics_df
# df["benchmark_return"]=benchmark_return
print('======查看绩效==========================')
print(result.metrics_df.round(2))
print('======查看订单==========================')
print(result.orders)
print('======查看持仓==========================')
print(result.positions)
print('======查看投资组合==========================')
print(result.portfolio)
print('======查看交易==========================')
print(result.trades)
create_strategy_charts(data_with_indicator,result,benchmark_return)

Loading bar data...
Loaded bar data: 0:00:03 

打印技术指标
           date  symbol     open     high      low    close     volume  \
0    2017-08-01  000852  7444.75  7456.41  7416.97  7452.93  116322120   
1    2017-08-02  000852  7445.29  7453.20  7385.35  7389.51  122436326   
2    2017-08-03  000852  7374.47  7451.61  7368.53  7424.11  114162783   
3    2017-08-04  000852  7417.31  7447.36  7389.15  7389.15  123600727   
4    2017-08-07  000852  7379.03  7445.68  7379.03  7445.18   99658819   
...         ...     ...      ...      ...      ...      ...        ...   
1756 2024-10-28  000852  5988.72  6082.09  5965.00  6082.09  326722938   
1757 2024-10-29  000852  6106.19  6113.64  5982.53  5983.84  369325299   
1758 2024-10-30  000852  5960.79  6043.59  5945.68  6019.35  324380186   
1759 2024-10-31  000852  6041.01  6161.56  6004.71  6116.66  386119485   
1760 2024-11-01  000852  6100.79  6125.41  5912.53  5930.23  409338276   

            macd  macdsignal   macdhist        MFI  sma_s

  0% (0 of 1515) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
 19% (291 of 1515) |####                 | Elapsed Time: 0:00:00 ETA:   0:00:00
 29% (451 of 1515) |######               | Elapsed Time: 0:00:00 ETA:   0:00:00
 53% (811 of 1515) |###########          | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (1515 of 1515) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


2019-07-17 00:00:00清仓
2019-10-10 00:00:00清仓
2020-10-28 00:00:00清仓
2020-12-08 00:00:00清仓
2021-01-19 00:00:00清仓
2021-01-29 00:00:00清仓
2021-12-27 00:00:00清仓

Finished backtest: 0:00:00
======基准收益率==========================
基准收益率12.330030767077934
======查看绩效==========================
                      name      value
0              trade_count      13.00
1     initial_market_value  500000.00
2         end_market_value  450386.73
3                total_pnl  -49613.27
4           unrealized_pnl      -0.00
5         total_return_pct      -9.92
6             total_profit  130431.82
7               total_loss -180045.09
8               total_fees     170.17
9             max_drawdown -117616.85
10        max_drawdown_pct     -21.17
11                win_rate      38.46
12               loss_rate      61.54
13          winning_trades       5.00
14           losing_trades       8.00
15                 avg_pnl   -3816.41
16          avg_return_pct      -0.57
17          avg_trade_bars      16.

In [ ]:
df[df['date'] == pb.param(name='bt_start_date')]["open"].values[0]
# df

,date,symbol,open,high,low,close,volume,macd,macdsignal,macdhist,MFI,sma_short,sma_long,sma250
0,2017-08-01,000852,7444.75,7456.41,7416.97,7452.93,116322120,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-02,000852,7445.29,7453.20,7385.35,7389.51,122436326,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-03,000852,7374.47,7451.61,7368.53,7424.11,114162783,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-04,000852,7417.31,7447.36,7389.15,7389.15,123600727,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-07,000852,7379.03,7445.68,7379.03,7445.18,99658819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756,2024-10-28,000852,5988.72,6082.09,5965.00,6082.09,326722938,311.094260,287.015788,24.078472,64.671347,5785.806,5205.862333,5329.79556
1757,2024-10-29,000852,6106.19,6113.64,5982.53,5983.84,369325299,305.636591,290.739949,14.896643,65.824494,5833.472,5255.049333,5329.68544
1758,2024-10-30,000852,5960.79,6043.59,5945.68,6019.35,324380186,300.710308,292.734021,7.976288,65.952273,5882.982,5306.510000,5330.13080
1759,2024-10-31,000852,6041.01,6161.56,6004.71,6116.66,386119485,301.186408,294.424498,6.761910,72.874968,5944.087,5360.812667,5331.22588
